In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pdfplumber
import re

In [336]:


# Path to your PDF file
pdf_path = "../data/pdf/Feb8.pdf"

with pdfplumber.open(pdf_path, pages = [1]) as pdf:
    # Extract text from each page
    text = ""
    for page in pdf.pages:
        text += page.extract_text();

#matches = re.findall(r'#(.*?)(?=TM)', text)

CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [337]:
matches = re.findall(r'Southern Oregon \d+..(#[\s\S]+?..TM Team)', text, re.DOTALL)
matches_date = re.findall(r'Date: \s*(........)', text)
matches

['# Player GS MIN FG 3PT FT ORB-DRB REB PF A TO BLK STL PTS\n12 Emma Schmerbach * 28 7-11 0-1 0-2 4-5 9 2 2 2 1 1 14\n20 Morgan Baird * 28 5-11 0-0 4-5 2-2 4 0 3 1 0 0 14\n10 Bridgette McIntyre * 30 4-8 4-8 0-0 0-2 2 2 1 2 0 1 12\n1 Meghan McIntyre * 32 2-10 0-1 6-10 2-5 7 0 4 2 0 3 10\n23 Clara Robbins * 11 0-1 0-0 0-2 0-1 1 4 0 0 1 3 0\n32 Eliza DiGiulio 24 5-8 1-2 1-2 1-1 2 1 3 0 1 1 12\n33 Mallory Williams 12 2-2 2-2 0-0 0-2 2 1 0 1 0 0 6\n5 Makayla Scurlock 12 1-3 0-0 0-0 0-4 4 1 0 1 0 1 2\n22 Sierra Scheppele 15 0-1 0-1 0-0 0-1 1 0 1 1 0 0 0\n13 Keeley Wright 8 0-1 0-0 0-0 1-0 1 1 1 0 0 0 0\nTM Team']

In [349]:
stats = matches[0]
stats

stats = stats.strip().split(sep = '\n')
split_stats = list(map(str.split, stats))

def int_convet(x:str):
    try:
        return int(x)
    except ValueError:
        return x
    
def merge_and_pop(lst:list):
    '''merges and pops the name'''
    lst[1] = lst[1] + ' ' + lst[2]
    lst.pop(2)
    return lst

def non_start(lst:list):
    '''fills in a value if the player didn't start'''
    if lst[2] != '*':
        lst = lst[:2] + ['-'] + lst[2:]
    return  lst

def get_opponent(raw_text:str):
    """
    The first line should hold something like
    'Southern Oregon (7-0, 1-0) -vs- College of Idaho (2-4, 0-1):'
    """
    line = raw_text.split('\n')[0]
    teams = line.split(' -vs- ')
    team1, team2 = re.findall(r'(^.+)(?: \(\S+\s\S+)', teams[0])[0], re.findall(r'(^.+)(?: \(\S+\s\S+)', teams[1])[0]

    if team1 == 'Southern Oregon':
            return team2
    return team1

opponent = get_opponent(text)
title = split_stats[0] + ['Opponent'] + ['Date']

player_stats = {key: [] for key in title}

players = split_stats[1:-1]
players = list(map(merge_and_pop, players))
players = list(map(non_start, players))
players

stat_dict = {key: [] for key in title}

for player in players:

    for stat, data in zip(title, player):
        stat_dict[stat].append(data)
    stat_dict['Opponent'].append(opponent)
    stat_dict['Date'].append(matches_date[0])


pd.DataFrame.from_dict(stat_dict)



,#,Player,GS,MIN,FG,3PT,FT,ORB-DRB,REB,PF,A,TO,BLK,STL,PTS,Opponent,Date
0,12,Emma Schmerbach,*,28,7-11,0-1,0-2,4-5,9,2,2,2,1,1,14,Corban (OR),02/08/25
1,20,Morgan Baird,*,28,5-11,0-0,4-5,2-2,4,0,3,1,0,0,14,Corban (OR),02/08/25
2,10,Bridgette McIntyre,*,30,4-8,4-8,0-0,0-2,2,2,1,2,0,1,12,Corban (OR),02/08/25
3,1,Meghan McIntyre,*,32,2-10,0-1,6-10,2-5,7,0,4,2,0,3,10,Corban (OR),02/08/25
4,23,Clara Robbins,*,11,0-1,0-0,0-2,0-1,1,4,0,0,1,3,0,Corban (OR),02/08/25
5,32,Eliza DiGiulio,-,24,5-8,1-2,1-2,1-1,2,1,3,0,1,1,12,Corban (OR),02/08/25
6,33,Mallory Williams,-,12,2-2,2-2,0-0,0-2,2,1,0,1,0,0,6,Corban (OR),02/08/25
7,5,Makayla Scurlock,-,12,1-3,0-0,0-0,0-4,4,1,0,1,0,1,2,Corban (OR),02/08/25
8,22,Sierra Scheppele,-,15,0-1,0-1,0-0,0-1,1,0,1,1,0,0,0,Corban (OR),02/08/25
9,13,Keeley Wright,-,8,0-1,0-0,0-0,1-0,1,1,1,0,0,0,0,Corban (OR),02/08/25
